<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/SVM_MNIST_Classifier/blob/master/SVM_MNIST_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SVM MNIST Classifier**

## **Libraries**

In [1]:
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## **Load MNIST Data**

In [2]:
# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## **Flatten the Images**

In [3]:
# Flatten the images
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test = X_test.reshape(-1, 28*28) / 255.0

## **Hyperparameters**

In [4]:
# Hyperparameters
batch_size = 1000
C = 1.0

## **Linear Kernel**

In [5]:
# Linear kernel
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

## **SVM Train for Binary Classification**

In [6]:
# SVM train for binary classification
def svm_train(X, y, C=1.0):
    n_samples, n_features = X.shape
    K = np.dot(X, X.T)

    P = matrix(np.outer(y, y) * K)
    q = matrix(-np.ones(n_samples))
    G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    A = matrix(y.astype('double'), (1, n_samples))
    b = matrix(0.0)

    sol = solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(sol['x'])

    sv = alphas > 1e-5
    alphas = alphas[sv]
    sv_X = X[sv]
    sv_y = y[sv]

    b = np.mean(sv_y - np.dot(sv_X, np.dot(sv_X.T, alphas * sv_y)))
    w = np.dot((alphas * sv_y), sv_X)

    return w, b

## **Predict**

In [7]:
# Predict
def svm_predict(X, w, b):
    return np.dot(X, w) + b